## Задание

#### Задание 1
Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:<br>
x - последовательность цифр<br>
y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10<br>

Задача:<br>
научить модель предсказывать y(i) по x(i)<br>
пробовать RNN, LSTM, GRU<br>

#### Задание 2 
(дополнительное и необязательное)
применить LSTM для решения лекционного практического задания

In [1]:
# импорт библиотек
import torch
from torch import nn
import random
import pandas as pd
from tqdm import tqdm
import time

#### Задание 1

#### 1. Подготовка данных

##### 1.1 Генерация

In [2]:
# функция для генерации данных
def gen_data(data_len): #принимает на вход длину данных
    X = torch.randint(0, 9, (data_len,)) # создаёт тензор из случайного набора цифр от 0 до 9 заданной длины
    Y = torch.zeros(data_len).type(torch.int64) # создаёт тензор заданной длины с нулями
    x1 = X[0]
    Y[0] = x1
    for i in range(1, len(X)): # заполняет тензо Y данными
        Y[i] = X[i] + x1
        if Y[i] >= 10:
            Y[i] -= 10
    return X,Y
X1,Y1 = gen_data(25)
X2,Y2 = gen_data(75)
X3,Y3 = gen_data(150)
X = [X1, X2, X3]
Y = [Y1, Y2, Y3]
X, Y

([tensor([4, 5, 2, 0, 5, 1, 5, 0, 6, 5, 6, 6, 2, 5, 8, 0, 8, 4, 5, 6, 3, 5, 6, 4,
          1]),
  tensor([0, 4, 0, 7, 8, 1, 1, 0, 4, 1, 3, 8, 0, 2, 2, 4, 5, 8, 0, 3, 5, 6, 7, 6,
          7, 5, 6, 8, 1, 2, 5, 5, 7, 4, 3, 2, 3, 4, 6, 3, 3, 7, 3, 1, 3, 1, 6, 7,
          7, 6, 7, 6, 1, 7, 6, 7, 2, 7, 7, 8, 7, 1, 3, 0, 8, 1, 5, 1, 0, 2, 7, 2,
          0, 3, 4]),
  tensor([1, 6, 4, 3, 5, 7, 1, 4, 3, 5, 7, 0, 4, 4, 1, 2, 0, 5, 0, 8, 4, 7, 1, 0,
          1, 5, 0, 3, 4, 3, 6, 6, 3, 6, 3, 4, 6, 2, 0, 1, 4, 8, 3, 4, 1, 2, 1, 6,
          5, 8, 1, 3, 8, 8, 3, 6, 4, 1, 0, 6, 7, 7, 0, 6, 0, 6, 0, 2, 2, 2, 5, 0,
          6, 8, 2, 4, 5, 4, 7, 1, 3, 1, 6, 4, 4, 6, 2, 7, 2, 3, 4, 2, 2, 6, 1, 6,
          1, 1, 5, 2, 6, 8, 4, 7, 6, 6, 0, 4, 6, 2, 5, 0, 8, 5, 0, 3, 2, 5, 0, 6,
          8, 6, 0, 5, 4, 4, 8, 2, 3, 4, 4, 6, 4, 8, 7, 7, 2, 2, 8, 7, 1, 7, 2, 8,
          8, 8, 2, 3, 8, 5])],
 [tensor([4, 9, 6, 4, 9, 5, 9, 4, 0, 9, 0, 0, 6, 9, 2, 4, 2, 8, 9, 0, 7, 9, 0, 8,
          5]),
  tensor([0, 4, 

In [3]:
# настройки
epoch = 1000          # количество эпох
dict_size = 10        # длина "словаря", в нашем случае 10 символов
batch_size = 5      # размер батча
embedding_size = 15 # размер эмбеддинга
hidden_size = 10    # размер скрытого состояния

In [4]:
# функция обучения и проверки модели
def test_model(model, optimizer, loss_data, accuracy_data, epochs=epoch):
    for i in range(3): # проходит по трём вариантам длины последовательности
        x = X[i]
        y = Y[i]
        print("Data length: {}".format(len(x)))
        for ep in range(epochs):
            train_loss = 0.
            train_passed = 0
            for i in range(int(len(x) / batch_size)):
                X_batch = x[i * batch_size: (i + 1) * batch_size]
                Y_batch = y[i * batch_size: (i + 1) * batch_size]
                optimizer.zero_grad()
                answers = model.forward(X_batch)
                loss = criterion(answers, Y_batch)
                train_loss += loss.item()
                loss.backward()
                optimizer.step()
                train_passed += 1
            if (ep+1) % 250 == 0:
                print("Epoch {}, Train loss: {:.3f}".format(ep+1, train_loss / train_passed))
        loss_data.append(train_loss / train_passed)
        accuracy = float((model(x).argmax(axis=1) == y).sum() / len(y))
        accuracy_data.append(accuracy)
        print("Accuracy: {:.3f}".format(accuracy))
        optimizer.zero_grad()
        for layer in model.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

##### 1.2 Создание класса сети

In [5]:
# создаём класс для сети
class NeuralNetwork(nn.Module):
    # параметры сети
    def __init__(self, 
                 rnnClass,                           # класс RNN, можем передавать любой, от этого будет зависеть, какая RNN ячейка будет создана
                 dict_size = dict_size,              # размер словаря
                 embedding_size = embedding_size,    # размер эмбеддинга
                 hidden_size = hidden_size           # размер скрытого состояния
                ):
        super().__init__()
        
        # в модели будем сохранять
        self.num_hiddens = hidden_size # размерность скрытого состояния
        self.embedding = nn.Embedding(dict_size, embedding_size) # слой эмбеддингов (на вход принимает размерность словаря и размерность эмбеддингов)
        # формируем два линейных слоя
        self.hidden = rnnClass(embedding_size, hidden_size, batch_first=True) # создаём RNN ячейку переданного в модель класса, на вход принимает
                                                                              # размер эмбеддинга, размер скрытого состояния и настройка, на каком месте в передаваемой 
                                                                              # последовательности стоит батч - на первом или на втором
        self.output = nn.Linear(hidden_size, dict_size) # последний линейный слой, который переводит размерность скрытого состояния в размерность словаря
        
    def forward(self, X): # функция forward принимает на вход батч элементов
        out = self.embedding(X) # прогоняем их через эмбеддинг, который возвращает преобразованные данные
        outputs, state = self.hidden(out) # отправляем эти преобразованные данные в слой скрытого состояния, на выход получаем outputs и state модели после
                                    # последнего элемента; в данном случае можем использовать state, т.к. предсказываем значение последнего элемента
                                    # outputs - скрытые состояния для каждого из 40 элементов
                                    # state - состояние модели после последнего элемента, которое показывает уверенность модели в каждом элементе словаря
        predictions = self.output(outputs)
        return predictions

In [6]:
# задаём функцию потерь и оптимизатор
criterion = torch.nn.CrossEntropyLoss()

#### 2. Обучение сетей

##### 2.1 RNN

In [7]:
RNN = NeuralNetwork(torch.nn.RNN)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(RNN.parameters(), lr=.001)
loss_RNN = []
accuracy_RNN = []
test_model(RNN, optimizer, loss_RNN, accuracy_RNN)
# loss_RNN, accuracy_RNN

Data length: 25
Epoch 250, Train loss: 1.472
Epoch 500, Train loss: 1.066
Epoch 750, Train loss: 0.819
Epoch 1000, Train loss: 0.652
Accuracy: 0.920
Data length: 75
Epoch 250, Train loss: 0.815
Epoch 500, Train loss: 0.392
Epoch 750, Train loss: 0.226
Epoch 1000, Train loss: 0.149
Accuracy: 1.000
Data length: 150
Epoch 250, Train loss: 0.434
Epoch 500, Train loss: 0.177
Epoch 750, Train loss: 0.110
Epoch 1000, Train loss: 0.082
Accuracy: 0.993


##### 2.2 LSTM

In [8]:
LSTM = NeuralNetwork(torch.nn.LSTM)
optimizer = torch.optim.SGD(LSTM.parameters(), lr=.001)
loss_LSTM = []
accuracy_LSTM = []
test_model(LSTM, optimizer, loss_LSTM, accuracy_LSTM)

Data length: 25
Epoch 250, Train loss: 2.143
Epoch 500, Train loss: 2.004
Epoch 750, Train loss: 1.870
Epoch 1000, Train loss: 1.730
Accuracy: 0.480
Data length: 75
Epoch 250, Train loss: 2.121
Epoch 500, Train loss: 1.911
Epoch 750, Train loss: 1.651
Epoch 1000, Train loss: 1.354
Accuracy: 0.760
Data length: 150
Epoch 250, Train loss: 1.758
Epoch 500, Train loss: 1.202
Epoch 750, Train loss: 0.734
Epoch 1000, Train loss: 0.432
Accuracy: 0.980


##### 2.3 GRU

In [9]:
GRU = NeuralNetwork(torch.nn.GRU)
optimizer = torch.optim.SGD(GRU.parameters(), lr=.001)
loss_GRU = []
accuracy_GRU = []
test_model(GRU, optimizer, loss_GRU, accuracy_GRU)

Data length: 25
Epoch 250, Train loss: 2.048
Epoch 500, Train loss: 1.825
Epoch 750, Train loss: 1.623
Epoch 1000, Train loss: 1.423
Accuracy: 0.600
Data length: 75
Epoch 250, Train loss: 1.725
Epoch 500, Train loss: 1.172
Epoch 750, Train loss: 0.739
Epoch 1000, Train loss: 0.468
Accuracy: 0.987
Data length: 150
Epoch 250, Train loss: 1.227
Epoch 500, Train loss: 0.568
Epoch 750, Train loss: 0.281
Epoch 1000, Train loss: 0.180
Accuracy: 0.993


#### 3. Результаты

In [10]:
data = {
    'RNN accuracy': accuracy_RNN,
    'RNN loss': loss_RNN,
    'LSTM accuracy': accuracy_LSTM,
    'LSTM loss': loss_LSTM,
    'GRU accuracy': accuracy_GRU,
    'GRU Loss' : loss_GRU
    }

# Преобразуйте словарь в DataFrame
df = pd.DataFrame(data)
df.index = ['25', '75', '150']
df.head()

,RNN accuracy,RNN loss,LSTM accuracy,LSTM loss,GRU accuracy,GRU Loss
25,0.920000,0.651972,0.48,1.729612,0.600000,1.422651
75,1.000000,0.148854,0.76,1.353828,0.986667,0.468191
150,0.993333,0.081718,0.98,0.432106,0.993333,0.179594


Обучал на 1000 эпох, чтобы достичь максимальных результатов. Как видно в ходе обучения точность и лосс улучшаются по мере роста эпох.
Сравнивая 3 модели по соотношению значения точности и лосса, лучший результат показывает RNN, на втором месте GRU, которая показывате неплохие результаты на длинной послеовательности, однако лосс у неё хуже. Самое низкое качество показывает LSTM, предполагаю, что это связано с простотой модели и данных.